In [12]:
import os
from dotenv import load_dotenv
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents import Tool


In [13]:
# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

In [14]:
# Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, max_tokens=1024, n=1)



In [15]:
# Read hdb resale file in csv format
df = pd.read_csv('./data/hdb_2024.csv')

df['tranc_year_month'] =pd.to_datetime(df['tranc_year_month'], format='%Y-%m')


In [16]:
# Create the agent with allow_dangerous_code=True
pd_agent = create_pandas_dataframe_agent(
                                        llm, 
                                        df, 
                                        verbose=True, 
                                        allow_dangerous_code=True, 
                                        agent_type=AgentType.OPENAI_FUNCTIONS,
                                        return_intermediate_steps=True, 
                                        max_iterations=20
                                    )

In [17]:
# Create the tool
pd_tool = Tool(
	name="Manipulate and Analyze tabular data with Code",
	func=pd_agent.invoke, # <-- This is the function that will be called when the tool is run. Note that there is no `()` at the end
	description="Useful for search-based queries",
)

In [18]:
#test pd tool
# 
prompt ="Max resale price of flat in clementi by flat type"

response = pd_tool.func(prompt)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "max_resale_price_clementi = df[df['town'] == 'CLEMENTI'].groupby('flat_type')['resale_price'].max() \nmax_resale_price_clementi"}`


flat_type
3 ROOM        820000.0
4 ROOM        993000.0
5 ROOM       1258000.0
EXECUTIVE    1068888.0
Name: resale_price, dtype: float64The maximum resale prices of flats in Clementi by flat type are as follows:

- 3 ROOM: 820,000
- 4 ROOM: 993,000
- 5 ROOM: 1,258,000
- EXECUTIVE: 1,068,888

> Finished chain.


In [7]:
def askHDBpanda(prompt):
    response = pd_agent.invoke(prompt)
    return response

In [22]:
my_dict = {
    "input": "What is max resale price for 5-room flats in Clementi?",
    "output": "The maximum resale price for 5-room flats in Clementi is $1,258,000.",
    "intermediate_steps": [...]
}

# Print the keys
print(my_dict.keys())

dict_keys(['input', 'output', 'intermediate_steps'])


In [28]:
response['output']

'The maximum resale price of a flat in Clementi is 1,258,000.'

In [19]:
import os
from dotenv import load_dotenv
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents import Tool

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=1024, n=1)

# Read HDB resale file in CSV format
df = pd.read_csv('./data/hdb_2024.csv')
df['tranc_year_month'] = pd.to_datetime(df['tranc_year_month'], format='%Y-%m')
def create_pd_agent(llm, df):
    """Create a pandas dataframe agent."""
    return create_pandas_dataframe_agent(
        llm=llm,
        df=df,
        verbose=True,
        allow_dangerous_code=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        return_intermediate_steps=True,
        max_iterations=20
    )

def ask_pd_agent(agent, prompt):
    """Invoke the pandas dataframe agent with a prompt."""
    try:
        response = agent.invoke(prompt)
        return response
    except Exception as e:
        return f"An error occurred: {e}"

# Create the agent
pd_agent = create_pd_agent(llm, df)

# Define the user prompt
user_prompt = "Provide summary statistics of the HDB resale dataset."

# Invoke the agent with the user prompt
response = ask_pd_agent(pd_agent, user_prompt)
print(response)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.describe(include='all')"}`


                     tranc_year_month       town flat_type block  \
count                            4744       4744      4744  4744   
unique                            NaN         26         6  1767   
top                               NaN  WOODLANDS    4 ROOM     5   
freq                              NaN        366      2060    14   
mean    2024-01-14 20:56:57.875210752        NaN       NaN   NaN   
min               2024-01-01 00:00:00        NaN       NaN   NaN   
25%               2024-01-01 00:00:00        NaN       NaN   NaN   
50%               2024-01-01 00:00:00        NaN       NaN   NaN   
75%               2024-02-01 00:00:00        NaN       NaN   NaN   
max               2024-02-01 00:00:00        NaN       NaN   NaN   
std                               NaN        NaN       NaN   NaN   

           street_name storey_range  floor_area_sqm flat_model  \

In [20]:
import os
from dotenv import load_dotenv
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from pydantic.v1 import BaseModel
from langchain_experimental.tools.python.tool import PythonAstREPLTool, PythonREPLTool
from langchain.agents import Tool

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
def execute_code():
    code = """\
    df = pd.read_csv('./data/hdb_2024.csv')
    df['tranc_year_month'] = pd.to_datetime(df['tranc_year_month'], format='%Y-%m')

    median_resale_price_5_room_clementi = df[(df['flat_type'] == '5 ROOM') & (df['town'] == 'CLEMENTI')]['resale_price'].median()
    """

    # Create an empty dictionary for the globals
    exec_globals = {}

    # Execute the code in the empty dictionary
    exec(code, exec_globals)

    # Return the result from the dictionary
    return exec_globals[user_prompt]

# Call function
median_resale_price_5_room_clementi = execute_code()

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/hdb_2024.csv')

# Group by 'Postal', 'Latitude', 'Longitude', 'flat_type', and 'town' and count the number of rows
grouped_df = df.groupby(['postal_code', 'latitude', 'longitude', 'flat_type', 'town']).size().reset_index(name='count')

grouped_df.columns

Index(['postal_code', 'latitude', 'longitude', 'flat_type', 'town', 'count'], dtype='object')

In [4]:
grouped_df.head()

,postal_code,latitude,longitude,flat_type,town,count
0,50034,1.286235,103.842303,3 ROOM,CENTRAL AREA,2
1,50532,1.284194,103.845986,3 ROOM,CENTRAL AREA,1
2,51335,1.282116,103.842846,3 ROOM,CENTRAL AREA,2
3,80003,1.277217,103.838704,5 ROOM,BUKIT MERAH,2
4,80005,1.276684,103.839459,5 ROOM,BUKIT MERAH,1
